In [ ]:
!wget https://zenodo.org/record/2582968/files/code4bench.rar?download=1

--2023-07-23 05:55:24--  https://zenodo.org/record/2582968/files/code4bench.rar?download=1
Resolving zenodo.org (zenodo.org)... 188.185.124.72
Connecting to zenodo.org (zenodo.org)|188.185.124.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 635883922 (606M) [application/octet-stream]
Saving to: ‘code4bench.rar?download=1’

code4bench.rar?down 100%[===================>] 606.43M  23.7MB/s    in 26s     

2023-07-23 05:55:51 (22.9 MB/s) - ‘code4bench.rar?download=1’ saved [635883922/635883922]



In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.1 MB/s eta 0:00:00


In [ ]:
import openai
openai.api_key = "YOUR OPENAI KEY"

def chat_with_gpt(prompt):
    try:
        response = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages = [{"role": "user", "content": prompt}],
        )
        message = response.choices[0].message.content
        return message
    except Exception as e:
        print("Error:", str(e))

In [ ]:
import re

def extract_code_blocks(text):
    code_blocks = []
    pattern = r"```(.*?)```"
    matches = re.findall(pattern, text, re.DOTALL)
    for match in matches:
        code_blocks.append(match.strip())
    return code_blocks

In [ ]:
gcd_str = """
def gcd(a, b):
    if b == 0:
        return a
    else:
        return gcd(a % b, b)
"""

In [ ]:
def get_code_intention(code_snippet):
    prompt = f"What is the intention of this code?\n\n{code_snippet}\n\nIntention:"
    intention = chat_with_gpt(prompt)
    return intention

In [ ]:
def generate_programs(intention):
    prompt = f"Generate two different programs between ``` ``` that have the intention without any print or testing and getting input and calling the funstions program1 and program2: {intention}\n\nProgram 1:"

    programs = extract_code_blocks(chat_with_gpt(prompt))
    # for i, program in enumerate(programs):
      # if 'print' in program:
      #   program = program.split('print')[1]
      # if 'gcd' in program:
      #   program = program.replace('gcd', f'program{i}')
    return programs

In [ ]:
code_snippets = generate_programs(intention)

In [ ]:
asb = []
for i, program in enumerate(code_snippets):
  asb.append(program.replace('gcd', f'program{i}'))

asb
# code_snippets

['def calculate_program0(a, b):\n    if b == 0:\n        return a\n    else:\n        return calculate_program0(b, a % b)',
 'def calculate_program1(a, b):\n    while b != 0:\n        temp = b\n        b = a % b\n        a = temp\n    return a']

In [ ]:
def generate_test_pool(code_snippet):
  prompt = f"""

  generate a test pool for this code\n\n{code_snippet} with input and output in list of python dict format like this between ``` ``` Test Pool:

    test_pool = [
      'input': 'a': num, 'b': num, 'output': num,
      'input': 'a': num, 'b': num, 'output': num,
    ]

  """
  response = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages = [{"role": "user", "content": prompt}],
    )
  test_cases = response.choices[0].message.content
  return test_cases

In [ ]:
c = generate_test_pool(gcd_str)

In [ ]:
c_ex = extract_code_blocks(c)

In [ ]:
result = c_ex[0].split('python')[1]

In [ ]:
exec(result)

In [ ]:
test_pool

[{'input': {'a': 15, 'b': 5}, 'output': 5},
 {'input': {'a': 20, 'b': 10}, 'output': 10},
 {'input': {'a': 8, 'b': 12}, 'output': 4},
 {'input': {'a': 100, 'b': 75}, 'output': 25},
 {'input': {'a': 7, 'b': 7}, 'output': 7},
 {'input': {'a': 0, 'b': 10}, 'output': 10}]

In [ ]:
gcd_str = """
def buggy(a, b):
    if b == 0:
        return a
    else:
        return buggy(a%b, b)
"""

exec(gcd_str)

for program in asb:
  exec(program)

test_cases = [
    {"input": [20, 30], "output": 10},
    {"input": [15, 45], "output": 15},
    {"input": [17, 23], "output": 1},
    {"input": [0, 5], "output": 5},
    {"input": [10, 0], "output": 10},
    {"input": [0, 0], "output": None},
    {"input": [100, 200], "output": 100},
    {"input": [123, 456], "output": 3},
    {"input": [-20, 30], "output": 10},
    {"input": [20, -30], "output": 10},
]

for i, test_case in enumerate(test_cases):
    expected_output_str = str(test_case['output'])
    out1 = calculate_program0(*test_case['input'])
    out2 = calculate_program1(*test_case['input'])
    try:
      out_buggy = buggy(*test_case['input'])
    except:
      out_buggy = 'exc'

    if out1 == out2:
      if out1 != out_buggy:
        print(out_buggy)
        print(test_case)
      else:
        continue
    else:
      continue

exc
{'input': [20, 30], 'output': 10}
exc
{'input': [15, 45], 'output': 15}
exc
{'input': [17, 23], 'output': 1}
exc
{'input': [0, 5], 'output': 5}
exc
{'input': [100, 200], 'output': 100}
exc
{'input': [123, 456], 'output': 3}
exc
{'input': [-20, 30], 'output': 10}
exc
{'input': [20, -30], 'output': 10}


In [24]:
import pandas as pd
df_submissions = pd.read_csv('data/py_submissions.csv')

df = df_submissions[(df_submissions['problems_id'] == 133) & (df_submissions['author'] == 9)]
df = df[['sourceCode']]
df.values.tolist()[1][0]

"s = list(map(int, input().split()))\r\nt = list(map(int, input().split()))\r\nxs = [ s[i] - t[i] for i in range(len(s)) ]\r\n\r\ndemand = []\r\nsupply = []\r\nfor x in xs:\r\n    if x < 0:\r\n        demand.append(-x)\r\n    else:\r\n        supply.append(x)\r\n\r\nres = True\r\nif len(demand):\r\n    res = sum(supply) >= sum(demand) * 2\r\n    if res and len(supply) == 2 and supply[0] % 2 == supply[1] % 2 == 1:\r\n        res = False\r\n\r\nprint('Yes' if res else 'No')"

In [44]:
df_submissions = pd.read_csv('data/py_submissions.csv')

df = df_submissions[(df_submissions['problems_id'] == 290) & (df_submissions['author'] == 9)]
df.head()

,Unnamed: 0,id,submission,sourceCode,author,memory,time,sent,countLine,problems_id,verdicts_id,languages_id,isduplicated
2,25,331321,25346581,"def main():\r\n\ta, b, c = map(int, input().sp...",9,4532,62,2017-03-08 20:28:48,10,290,1,8,0
3,26,331324,25346921,"def d():\r\n a, b, c = map(int,input().spli...",9,4516,62,2017-03-08 20:47:39,20,290,2,8,0


In [38]:
df_testcases = pd.read_csv('data/py_testcases.csv')
df_testcases = df_testcases[df_testcases['problems_id'] == 133]
a = df_testcases[['expectedresult', 'inputdata']].to_dict('records')


In [39]:
import random
t = random.choice(a)

In [40]:
t

{'expectedresult': 'No',
 'inputdata': '500000 999999 500000\r\r\n1000000 0 500000'}

In [46]:
df_submissions = pd.read_csv('data/py_submissions.csv')
df_submissions.iloc[2].sourceCode

'def main():\r\n\ta, b, c = map(int, input().split())\r\n\tmin_a = min(a , b)\r\n\tmax_a = max(a, b)\r\n\tfor i in range(c // min_a + 1):\r\n\t\tif (c - min_a * i) % max_a == 0:\r\n\t\t\treturn "Yes"\r\n\treturn "No"\r\n\r\nprint(main())'